In [19]:
import pinecone
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, create_react_agent
from langchain.agents import AgentExecutor

In [3]:
# Load environment variables
load_dotenv()

True

# Setting up Tools

In [4]:
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
index = pc.Index("restaurant-info-medium-testing")

In [5]:
embeddings_model = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'),
                                        model="text-embedding-3-small")

In [6]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings_model)

In [7]:
search = TavilySearchResults(max_results=3)

In [8]:
search.invoke("Where is sushi in Los Angeles")

[{'url': 'https://www.latimes.com/food/list/the-best-sushi-bars-in-los-angeles',
  'content': 'For eight years, from 2012 to 2020, Shunji occupied the wildest, most only-in-Los-Angeles location for a sushi bar possible: a stuccoed 1930s-era building shaped like a soup bowl.'},
 {'url': 'https://www.yelp.com/search?find_desc=sushi&find_loc=Los+Angeles,+CA',
  'content': 'Top 10 Best sushi Near Los Angeles, California. 1. Yeon Omakase and Sushi. 2. Izakaya Osen - Los Angeles. 3. Chubby Cattle Little Tokyo. 4. Hato Sushi.'},
 {'url': 'https://la.eater.com/maps/los-angeles-best-sushi-restaurants-omakase',
  'content': 'Soko also offers beautiful bento takeout and delivery for those wanting quality sushi at home or in the office. Open in Google Maps. Foursquare. 101 Wilshire Blvd, Santa Monica, CA 90401. (310 ...'}]

In [9]:
retriever = vector_store.as_retriever(k=1)

In [10]:
retriever.invoke("where is sushi")[0]

Document(id='fc165d3d-c44c-470c-9217-b83cfe78a4cf', metadata={'address': '116 Japanese Village Plaza Mall, Los Angeles, CA 90012, USA', 'name': 'Sushi & Teri', 'place_id': 'ChIJizyr7UfGwoARt2VuQwQ0eIw', 'rating': 4.1}, page_content='Sushi & Teri is a small, inexpensive Japanese restaurant located in Little Tokyo, offering a variety of sushi rolls, ramen, miso soup, and cucumber salad. Customers generally feel that the food is decent but not exceptional, with notable mentions of the Alaska and Baja rolls, Washington roll, ramen served piping hot, and the cucumber salad. Key pros include friendly service, affordable prices, and the convenience of ordering via tablet, while cons revolve around long wait times, understaffing issues, and smaller portion sizes compared to nearby restaurants.')

In [11]:
retriever_tool = create_retriever_tool(
    retriever,
    "restaurant_search",
    "Search for information about restaurants in Los Angeles. If user prompts for restauraunts, USE THIS TOOL BY DEFAULT!",
)

In [12]:
tools = [search, retriever_tool]

In [13]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [14]:
llm.invoke("where is lebron currently", temperature=0.1)

AIMessage(content='As of October 2023, LeBron James is playing for the Los Angeles Lakers in the NBA. If you need more specific information or updates, please let me know!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 12, 'total_tokens': 47, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-c73d3c57-7ce4-45fb-b6e2-fcf593d08612-0', usage_metadata={'input_tokens': 12, 'output_tokens': 35, 'total_tokens': 47})

In [15]:
prompt = hub.pull("hwchase17/openai-functions-agent")

## Creating Agent + Executor

In [16]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [17]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
agent_executor.invoke({"input": "where is sushi gen and what food does it have?"})



> Entering new AgentExecutor chain...

Invoking: `restaurant_search` with `{'query': 'Sushi Gen Los Angeles'}`


Sushi Gen is a well-regarded sushi restaurant in LA known for its fast service and high-quality, fresh fish. Customers rave about specific items such as the toro (fatty tuna), fatty salmon, sashimi deluxe, and the $23 lunch special, which is considered a great value. While many patrons appreciate the quality and service, some express disappointment over long wait times, inconsistent experiences with certain menu items like the deluxe sushi set, and the restaurant's outdated decor. The price range is considered expensive. Overall, the sentiment is largely positive, with a few notable criticisms.

Sushi Kaneyoshi in LA is a highly praised sushi restaurant offering an omakase experience that includes dishes like oyster, monkfish liver (ankimo), mackerel handroll, and tamago. Customers generally feel positively about the restaurant, appreciating the fresh fish, authentic Japan

{'input': 'where is sushi gen and what food does it have?',
 'output': "Sushi Gen is a well-regarded sushi restaurant located in Los Angeles. It is known for its fast service and high-quality, fresh fish. Some popular menu items include:\n\n- **Toro (fatty tuna)**\n- **Fatty salmon**\n- **Sashimi deluxe**\n- **$23 lunch special**, which is considered a great value\n\nWhile many customers appreciate the quality and service, some have noted long wait times and occasional inconsistencies with certain menu items, particularly the deluxe sushi set. The restaurant's decor has been described as somewhat outdated, and the price range is considered expensive. Overall, the sentiment towards Sushi Gen is largely positive, with a few criticisms."}

## Creating reACT Agent

In [30]:
prompt = hub.pull("hwchase17/react")
print(prompt.template)


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [31]:
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [32]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [34]:
agent_executor.invoke({"input": "what is sushi gen and what type of food is it? also give me the address"})



> Entering new AgentExecutor chain...
I need to find information about Sushi Gen, including what type of food it serves and its address. Since this is about a restaurant, I will use the restaurant search tool.

Action: restaurant_search  
Action Input: 'Sushi Gen Los Angeles'  Sushi Gen is a well-regarded sushi restaurant in LA known for its fast service and high-quality, fresh fish. Customers rave about specific items such as the toro (fatty tuna), fatty salmon, sashimi deluxe, and the $23 lunch special, which is considered a great value. While many patrons appreciate the quality and service, some express disappointment over long wait times, inconsistent experiences with certain menu items like the deluxe sushi set, and the restaurant's outdated decor. The price range is considered expensive. Overall, the sentiment is largely positive, with a few notable criticisms.

Sushi Kaneyoshi in LA is a highly praised sushi restaurant offering an omakase experience that includes dishes like o

{'input': 'what is sushi gen and what type of food is it? also give me the address',
 'output': "Sushi Gen is a well-regarded sushi restaurant in Los Angeles known for its high-quality, fresh fish, offering classic sushi dishes. The address is typically located in the Little Tokyo area of Los Angeles. However, I recommend checking a reliable map or restaurant directory for the exact address, as it wasn't specified in the search results."}

In [35]:
agent_executor.invoke({"input": "what is the weather in california right now"})



> Entering new AgentExecutor chain...
I should search for the current weather conditions in California. 
Action: tavily_search_results_json
Action Input: "current weather in California" [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'California City', 'region': 'California', 'country': 'United States of America', 'lat': 35.13, 'lon': -117.99, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1727908180, 'localtime': '2024-10-02 15:29'}, 'current': {'last_updated_epoch': 1727907300, 'last_updated': '2024-10-02 15:15', 'temp_c': 40.5, 'temp_f': 104.9, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 174, 'wind_dir': 'S', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 8, 'cloud': 0, 'feelslike_c': 40.7, 'feelslike_f': 105.2, 'windchill_c': 39.3, 'windchill_f': 102.7, 'heatindex_c': 38.9, 'heatindex_

{'input': 'what is the weather in california right now',
 'output': 'The weather in California right now is sunny with a temperature of approximately 104.9°F (40.5°C) and low humidity at 8%.'}